In [ ]:
#Imports
%matplotlib inline
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt
import os
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout
import pydot

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #INFO and WARNING messages are not printed
import time

In [ ]:


# Load Transactions Data
interactions=pd.read_csv('HM_interactions.csv')
interactions

In [ ]:
#Test set creation

# convert timestamp to datetime
interactions['timestamp'] = pd.to_datetime(interactions['timestamp'], unit='s')
# sort by date, group by customer_id and choose the most recent
interactions_sorted = interactions.sort_values(['timestamp'],ascending=True).groupby('customer_id').tail(1)
#  we create the test_set with the most recent purchase
test_set = interactions_sorted[['customer_id', 'article_id']].copy()
# add "interactions" column
test_set['interaction']='1'
test_set

In [ ]:
# Train set creation

# rows to be dropped are selectect (test_set) 
rows_to_drop = list(test_set.index.values)
# rows are dropped
pre_train_set = interactions.drop(rows_to_drop)
# order interactions by clients
pre_train_set = pre_train_set.sort_values(by=['customer_id'])
#we select "customer_id" and "article_id" columns and add interactions column
pre_train_set = pre_train_set[['customer_id', 'article_id']].copy()
# add interaction column
pre_train_set['interaction'] = 1
pre_train_set

In [ ]:
# Negative sampling

#transform to numpy for sampling
trans_matrix=pd.DataFrame(pre_train_set).to_numpy()

# Client's list
customer_list=np.array(np.unique(trans_matrix[:,0]))
customer_list

In [ ]:
# negative sampling loop
negative_sampling_all=np.empty((0, 3), int)
for i in range(np.size(customer_list)):
  # save all articles bought by the i-th client
  find=trans_matrix[np.where(trans_matrix[:,0] == customer_list[i])][:,1]
  # we save 4 random articles not bought by the i-th customer
  notfind=np.random.choice(trans_matrix[trans_matrix[:,0]!=i][:,1],4*np.size(find))
  create_notfind=np.column_stack((np.transpose(np.full(np.size(trans_matrix[trans_matrix[:,0]==customer_list[i]][:,0])*4,customer_list[i])), np.transpose(notfind),np.zeros(np.size(trans_matrix[trans_matrix[:,0]==customer_list[i]][:,0])*4)))
  # train_set generation
  negative_sampling=np.concatenate((trans_matrix[trans_matrix[:,0]==customer_list[i]], create_notfind))
  negative_sampling_all = np.concatenate((negative_sampling_all,negative_sampling))

In [ ]:
# convert to dataframe
train_set = pd.DataFrame(negative_sampling_all, columns = ['customer_id','article_id','interaction'])
train_set=train_set.astype(int)
customer0=train_set[train_set.customer_id==0]
customer0[customer0.interaction==0]
train_set

In [ ]:
# Load model from h5 file

my_model = keras.models.load_model('my_model/my_model_12_d')
my_model.load_weights("my_model/weights_12_d.h5")

In [ ]:
#Model implementation with keras

N = interactions.customer_id.unique() #Number of customers in the data
M = interactions.article_id.unique() #Number of articles in the data
n_users, n_item = len(N), len(M)

n_latent_factors = 64 #Latent factors; K must be a power of two

#articles input
item_input = keras.layers.Input(shape=[1],name='Item')
#articles embedding
item_embedding = keras.layers.Embedding(n_item + 1, n_latent_factors, name='Item-Embedding')(item_input)
#articles vector
item_vec = keras.layers.Flatten(name='FlattenItems')(item_embedding)
#customer input
user_input = keras.layers.Input(shape=[1],name='User')
#customer embedding
user_embedding = keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input)
#customer vector
user_vec = keras.layers.Flatten(name='FlattenUsers')(user_embedding)
#dot product
prod = keras.layers.dot([item_vec, user_vec], axes=1,name='DotProduct')
#add dropout to avoid overfitting
drop = keras.layers.Dropout(0.2, name='Dropout')(prod)
#add Sigmoid activation function to normalize outputs from 0 to 1
act = keras.layers.Dense(1, activation='sigmoid', name='Activation')(drop)

#generate the model
model = keras.Model([user_input, item_input], drop)

In [ ]:
#compile the model
#Adam Optimizer uses GD; BCE as loss function 
model.compile(optimizer='Adam', loss='binary_crossentropy')
#visualize model
tf.keras.utils.plot_model(model, to_file='model.png')

In [ ]:
start_time = time.time()
#train the model
history = model.fit([train_set.customer_id, train_set.article_id], train_set.interaction, epochs=12, verbose=0)
pd.Series(history.history['loss']).plot(logy=True)
plt.xlabel("Epoch")
plt.ylabel("Training Error")

end_time = time.time()
 
print("The time of execution of above program (in minutes) is :", (end_time - start_time)/60)

In [ ]:
# Save keras model in h5 file
# model.save("modelname")
# model.save_weights("weightsname.h5")

In [ ]:
#Example of one learnt article embedding with k latent factors
article_embedding_learnt = model.get_layer(name='Item-Embedding').get_weights()[0]
pd.DataFrame(article_embedding_learnt).describe()

In [ ]:
#Example of one learnt customer embedding with k latent factors
customer_embedding_learnt = model.get_layer(name='User-Embedding').get_weights()[0]
pd.DataFrame(customer_embedding_learnt).describe()

In [ ]:
#Evaluation of the recommender model: 

In [ ]:
#We create a function that outputs the best recommended articles to buy for a given user not taking into account articles that the customer has already bought. 
def recommend(user_id, number_of_articles=50):
  #Ranking value of each article associated by the model 
  articles = customer_embedding_learnt[user_id]@article_embedding_learnt.T 
  #Obtain the first 50 articles with highest value
  mids = np.argpartition(articles, -number_of_articles)[-number_of_articles:]
  #Get the articles that are already bought by each customer
  art_buy=np.array(pre_train_set[pre_train_set['customer_id']==user_id].article_id)
  #Get the first 10 articles without considering the ones that are already bought. 
  mids_not_buy = [i for i in mids if i not in art_buy][0:10]
  return mids_not_buy

In [ ]:
#Function that checks if the last bought article (test_set) belongs to the recommended list of top10 articles. 
def heat_ratio_recommend(df):
  exito_recommend=[]
  for column in df['customer_id'].unique().tolist():
    #obtains the top10 articles of each customer
    candidates = recommend(column)
    #Checks if the article from the test_set is in the candidates list
    if df[df.customer_id == column].article_id.tolist() in candidates: 
      exito_recommend.append(1)
    else:
      exito_recommend.append(0)
  return(exito_recommend) 

In [ ]:
#HR from the recommendation based on top10 scores for each customer: mean of succes. 
hr_recommend=np.mean(heat_ratio_recommend(test_set))
hr_recommend

In [ ]:
#Concatenate all recommended articles in one array. 
recommend_model = []
for x in range(np.size(customer_list)):
   myArray = recommend(x)
   recommend_model += myArray

In [ ]:
#Calculate the coverage to see how many articles have been recommended by the model over the total. 
cov_recommend=(np.size(np.unique(recommend_model))/np.size(np.unique(pre_train_set.article_id)))*100
cov_recommend

In [ ]:
#Evaluation of the random recommender: 

In [ ]:
#We create a function that outputs 10 articles randomly not taking into account articles that the customer has already bought. 
 #list of all articles
all_art = np.unique(train_set.article_id)
def random(user_id):
  #we select 50 randomly
  art_random =np.random.choice(all_art, 50)
  #check the articles already bought by the customer
  art_buy=np.array(pre_train_set[pre_train_set['customer_id']==user_id].article_id)
  #select 10 of the randoms without considering the ones in art_buy
  mids_random = [i for i in art_random if i not in art_buy][0:10]
  return mids_random

In [ ]:
#Join all randomly recommended articles for all customers 
random_recommend = []
for x in range(np.size(customer_list)):
   myArray = random(x)
   random_recommend += myArray

In [ ]:
#Calculate the coverage to see how many articles have been recommended randomly over the total. 
cov_random=(np.size(np.unique(random_recommend))/np.size(np.unique(pre_train_set.article_id)))*100
cov_random

In [ ]:
#Evaluate results: HEAT RATIO for random recommendation: 
def heat_ratio_random(df):
  exito_random=[]
  for column in df['customer_id'].unique().tolist():
    #In this case candidates come from the articles recommended randomly by the random function. 
    candidates = random(column)
    if df[df.customer_id == column].article_id.tolist() in candidates:
      exito_random.append(1)
    else:
      exito_random.append(0)
  return(exito_random) 

In [ ]:
#HR from the recommendation based on the 10 randomly choosen articles
hr_random=np.mean(heat_ratio_random(test_set))
hr_random

In [ ]:
#Evaluation of the popularity recommender: we recommend to all customers the most popular articles. 
interactions.head()

In [ ]:
# Get top 10 most bought articles --> most populars
# we drop all interactions that appear several times on a given customer
pre_train_unique=[]
for i in range(np.size(customer_list)):
    articles_unique=pre_train_set[pre_train_set.customer_id==i].article_id.drop_duplicates().tolist()
    pre_train_unique += articles_unique


In [ ]:
#conversion to dataframe
pre_train_unique=pd.DataFrame(pre_train_unique, columns = ['article_id'])
pre_train_unique

In [ ]:
# Output -> 10 most bought articles
n = 10
mids_popular_unique=pre_train_unique['article_id'].value_counts()[:n].index.tolist()
mids_popular_unique

In [ ]:
#Evaluate results: HEAT RATIO for common recommendations --> random or popular
def heat_ratio_common(df,mids):
  exito=[]
  #It checks the test_set with the 10 most popular articles (all equal for all customers)
  for column in df['customer_id'].unique().tolist():
    if all(item in mids for item in df[df.customer_id == column].article_id.tolist()):
      exito.append(1)
    else:
      exito.append(0)
  return(exito) 

In [ ]:
# #HR from the recommendation based on the 10 most popular recommended articles
hr_popularity=np.mean(heat_ratio_common(test_set,mids_popular_unique))
hr_popularity

In [ ]:
#Calculate the coverage to see how many articles have been recommended over the total considering the ten most popular articles.  
cov_popular=(np.size(mids_popular_unique)/np.size(np.unique(pre_train_set.article_id)))*100
cov_popular